In [1]:
from stitching import *
import numpy as np
import cv2 as cv
import os
import pandas as pd

import scipy.stats

from coordinate_transform import findImageResult

In [2]:
outer_folder = R"C:\data\kart"
names = [
        "Dallas",
        "Finnmark",
        "Hellas",
        "Istanbul",
        "Tromsoe",
        "Trondheim"
    ]

split = [250, 400, 500]
def getFolderName(name, split_num):
    return outer_folder + "\\" + name + "\\" + f"split_{split_num}_{split_num}"



In [3]:
results = []
result_names = []
result_split = []
for name in names:
    for s in split:
        folder = getFolderName(name, s)
        print(folder)
        result_y, true_y, result_x, true_x = stitchFolder(folder, altered=False)
        d = {
            "name": name,
            "split": s,
            "result_y": result_y,
            "result_x": result_x,
            "true_y": true_y,
            "true_x": true_x 
        }
        results.append(d)
        result_names.append(name)
        result_split.append(s)

result_names = np.array(result_names)
result_split = np.array(result_split)

C:\data\kart\Dallas\split_250_250
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : Tr

c:\Users\sondr\OneDrive - NTNU\KodeTesting\python\image_stitching\frequency\frequency.py:7: RuntimeWarning: invalid value encountered in divide
  return np.where(divisor == 0, 0, dividend/divisor)


Correlation shape is equal to self shape : True
10
C:\data\kart\Hellas\split_400_400
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
C:\data\kart\Hellas\split_500_500
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
C:\data\kart\Istanbul\split_250_250
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to self shape : True
10
10
Correlation shape is equal to se

In [4]:
def extractToDf(results):
    """
    Extracts the data from results in to a dataframe like 

    INDEX, origin_image, sub_image_width, sub_image_height, avg_results: (normal, template, frequency, SIFT, ORB, AKAZE, KAZE)(both x and y), typetall result(all and x and y )


    
    """

    out_dict = {}
    out_dict_plus = {}

    for element in results:
        if "original" not in out_dict:
            out_dict["original"] = [element["name"]]
            out_dict["sub_width"] = [element["result_y"][0, 0]["image1"].shape[1]]
            out_dict["sub_height"] = [element["result_y"][0, 0]["image1"].shape[0]]
            out_dict["shape"] = [[element["result_x"].shape[0], element["result_y"].shape[1]]]
            out_dict_plus["original"] = [element["name"]]
            out_dict_plus["sub_width"] = [element["result_y"][0, 0]["image1"].shape[1]]
            out_dict_plus["sub_height"] = [element["result_y"][0, 0]["image1"].shape[0]]
            out_dict_plus["shape"] = [[element["result_x"].shape[0], element["result_y"].shape[1]]]
            
        else:
            out_dict["original"].append(element["name"])
            out_dict["sub_width"].append(element["result_y"][0, 0]["image1"].shape[1])
            out_dict["sub_height"].append(element["result_y"][0, 0]["image1"].shape[0])
            out_dict["shape"].append([element["result_x"].shape[0], element["result_y"].shape[1]])
            out_dict_plus["original"].append(element["name"])
            out_dict_plus["sub_width"].append(element["result_y"][0, 0]["image1"].shape[1])
            out_dict_plus["sub_height"].append(element["result_y"][0, 0]["image1"].shape[0])
            out_dict_plus["shape"].append([element["result_x"].shape[0], element["result_y"].shape[1]])



        methods = ["normal", "template", "frequency", "SIFT", "ORB", "AKAZE", "KAZE"]
        for r, t in zip(["result_x", "result_y"], ["true_x", "true_y"]):
            if r == "result_x":
                direction = "right"
            else:
                direction = "up"
            result = element[r]
            true = element[t]
            
            
            failed = {}

            x = {}
            y = {}

            diff_expected = {}

            x_plus = {}
            y_plus = {}

            time_eval = {}

            for keys in methods:
                x[keys] = np.zeros(result.shape[:2], int)
                y[keys] = np.zeros(result.shape[:2], int)
                x_plus[keys] = np.zeros(result.shape[:2], int)
                y_plus[keys] = np.zeros(result.shape[:2], int)
                diff_expected[keys] = np.zeros(result.shape[:2], np.float16)
                failed[keys] = np.full(result.shape[:2], False, bool)

                time_eval[keys] = np.full(result.shape[:2], 0, float)

            relative_pos = np.full((result.shape[0], result.shape[1], 2), np.float32)
            expected_pos = np.full((result.shape[0], result.shape[1], 2), np.float32)

            relative_pos = np.array([0, 0])
            expected_pos = np.array([0, 0])
            true_pos = np.array([0, 0])


            for i in range(result.shape[0]):
                for j in range(result.shape[1]):
                    relative_position = result[i, j]["relative position"]
                    relative_pos[0] += relative_position[0]
                    relative_pos[1] += relative_position[1]
                    expected_position = result[i, j]["expected position"]
                    expected_pos[0] += expected_pos[0]
                    expected_pos[1] += expected_pos[1]
                    true_pos[0] += true[i, j][0]
                    true_pos[1] += true[i, j][1]
                    for method in methods:
                        this_pos = relative_position - result[i, j][method].getResult()
                        x[method][i, j] = true[i, j][1] - this_pos[1]
                        y[method][i, j] = true[i, j][0] - this_pos[0]
                        x_plus[method][i, j] = true[i, j][1] + this_pos[1]
                        y_plus[method][i, j] = true[i, j][0] + this_pos[0]
                        diff_expected[method][i, j] = np.sum(np.abs(expected_position - result[i, j][method].getResult()))/2 

                        failed[method][i, j] = result[i, j][method].has_failed

                        if result[i, j][method].getTime() is None:
                            print(f"Time is none for {method} [{i}] {j}")
                            time_eval[method][i, j] = np.inf
                        elif result[i, j][method].has_failed:
                            time_eval[method][i, j] = np.inf
                        else:
                            t = result[i, j][method].getTime()
                            time_eval[method][i, j] = t
                            # print(time_eval[method][i, j], t, time_eval[method][i, j] == result[i, j][method].getTime(), result[i, j][method].has_failed)
                    


            true_pos = true_pos/(result.shape[0]*result.shape[1])
            expected_pos = expected_pos/(result.shape[0]*result.shape[1])
            relative_pos = relative_pos/(result.shape[0]*result.shape[1])
    
            if f"{direction}_true" not in out_dict:
                out_dict[f"{direction}_true"] = []
                out_dict[f"{direction}_relative"] = []
                out_dict[f"{direction}_expect"] = []

            out_dict[f"{direction}_true"].append(true_pos)
            out_dict[f"{direction}_relative"].append(relative_pos)
            out_dict[f"{direction}_expect"].append(expected_pos)



            # out_dict[f"{direction}_true"].append([np.average(true[:, :, 0]), np.average(true[:, :, 1])])
            # out_dict[f"{direction}_relative"].append([np.average(relative_pos[:, :, 0]), np.average(relative_pos[:, :, 1])])
            # out_dict[f"{direction}_expect"].append([np.average(expected_pos[:, :, 0]), np.average(expected_pos[:, :, 1])])
            # print([np.average(true[:, :, 0]), np.average(true[:, :, 1])])
            # print([np.average(relative_pos[:, :, 0]), np.average(relative_pos[:, :, 1])])
            # print([np.average(expected_pos[:, :, 0]), np.average(expected_pos[:, :, 1])])



            for method in methods:
    
                if f"{method}_{direction}_failed" not in out_dict:
                    out_dict[f"{method}_{direction}_time"] = []
                    out_dict[f"{method}_{direction}_time_max"] = []
                    out_dict[f"{method}_{direction}_time_min"] = []
                    out_dict[f"{method}_{direction}_failed"] = []
                    out_dict[f"{method}_{direction}_x_avg"] = []
                    out_dict[f"{method}_{direction}_y_avg"] = []
                    out_dict[f"{method}_{direction}_x_mode"] = []
                    out_dict[f"{method}_{direction}_y_mode"] = []
                    out_dict[f"{method}_{direction}_x_max"] = []
                    out_dict[f"{method}_{direction}_x_min"] = []
                    out_dict[f"{method}_{direction}_x_max_arg"] = []
                    out_dict[f"{method}_{direction}_x_min_arg"] = []
                    out_dict[f"{method}_{direction}_y_max"] = []
                    out_dict[f"{method}_{direction}_y_min"] = []
                    out_dict[f"{method}_{direction}_y_max_arg"] = []
                    out_dict[f"{method}_{direction}_y_min_arg"] = []
                    out_dict_plus[f"{method}_{direction}_x_avg"] = []
                    out_dict_plus[f"{method}_{direction}_y_avg"] = []
                    out_dict_plus[f"{method}_{direction}_x_mode"] = []
                    out_dict_plus[f"{method}_{direction}_y_mode"] = []
                    out_dict_plus[f"{method}_{direction}_x_max"] = []
                    out_dict_plus[f"{method}_{direction}_x_min"] = []
                    out_dict_plus[f"{method}_{direction}_x_max_arg"] = []
                    out_dict_plus[f"{method}_{direction}_x_min_arg"] = []
                    out_dict_plus[f"{method}_{direction}_y_max"] = []
                    out_dict_plus[f"{method}_{direction}_y_min"] = []
                    out_dict_plus[f"{method}_{direction}_y_max_arg"] = []
                    out_dict_plus[f"{method}_{direction}_y_min_arg"] = []


                time_full = time_eval[method][np.where(time_eval[method] != np.inf)]

                if len(time_full) != 0:
                    out_dict[f"{method}_{direction}_time"].append(np.average(time_full))
                    out_dict[f"{method}_{direction}_time_max"].append(np.amax(time_full))
                    out_dict[f"{method}_{direction}_time_min"].append(np.amin(time_full))
                else:
                    out_dict[f"{method}_{direction}_time"].append(0)
                    out_dict[f"{method}_{direction}_time_max"].append(0)
                    out_dict[f"{method}_{direction}_time_min"].append(0)

                out_dict[f"{method}_{direction}_failed"].append(np.sum(failed[method]))
                # out_dict[f"{method}_{direction}_failed"].append(np.array(failed[method]))

                # out_dict[f"{method}_{direction}_x_avg"].append(np.array(x[method]))
                # out_dict[f"{method}_{direction}_y_avg"].append(np.array(y[method]))
                out_dict[f"{method}_{direction}_x_avg"].append(np.average(np.array(x[method])))
                out_dict[f"{method}_{direction}_y_avg"].append(np.average(np.array(y[method])))
                mode_x, _ = scipy.stats.mode(x[method].flatten())
                mode_y, _ = scipy.stats.mode(y[method].flatten())
                out_dict[f"{method}_{direction}_x_mode"].append(mode_x)
                out_dict[f"{method}_{direction}_y_mode"].append(mode_y)

                out_dict[f"{method}_{direction}_x_max"].append(np.amax(x[method]))
                out_dict[f"{method}_{direction}_x_min"].append(np.amin(x[method]))
                out_dict[f"{method}_{direction}_x_max_arg"].append(np.unravel_index(np.argmax(x[method]), result.shape[:2]))
                out_dict[f"{method}_{direction}_x_min_arg"].append(np.unravel_index(np.argmin(x[method]), result.shape[:2]))
                out_dict[f"{method}_{direction}_y_max"].append(np.amax(y[method]))
                out_dict[f"{method}_{direction}_y_min"].append(np.amin(y[method]))
                out_dict[f"{method}_{direction}_y_max_arg"].append(np.unravel_index(np.argmax(y[method]), result.shape[:2]))
                out_dict[f"{method}_{direction}_y_min_arg"].append(np.unravel_index(np.argmin(y[method]), result.shape[:2]))
                



                out_dict_plus[f"{method}_{direction}_x_avg"].append(np.average(x_plus[method].flatten()) )
                out_dict_plus[f"{method}_{direction}_y_avg"].append(np.average(y_plus[method].flatten()) )
                mode_x, _ = scipy.stats.mode(x_plus[method].flatten())
                mode_y, _ = scipy.stats.mode(y_plus[method].flatten())
                out_dict_plus[f"{method}_{direction}_x_mode"].append(mode_x)
                out_dict_plus[f"{method}_{direction}_y_mode"].append(mode_y)

                out_dict_plus[f"{method}_{direction}_x_max"].append(np.amax(x_plus[method]))
                out_dict_plus[f"{method}_{direction}_x_min"].append(np.amin(x_plus[method]))
                out_dict_plus[f"{method}_{direction}_x_max_arg"].append(np.unravel_index(np.argmax(x_plus[method]), result.shape[:2]))
                out_dict_plus[f"{method}_{direction}_x_min_arg"].append(np.unravel_index(np.argmin(x_plus[method]), result.shape[:2]))
                out_dict_plus[f"{method}_{direction}_y_max"].append(np.amax(y_plus[method]))
                out_dict_plus[f"{method}_{direction}_y_min"].append(np.amin(y_plus[method]))
                out_dict_plus[f"{method}_{direction}_y_max_arg"].append(np.unravel_index(np.argmax(y_plus[method]), result.shape[:2]))
                out_dict_plus[f"{method}_{direction}_y_min_arg"].append(np.unravel_index(np.argmin(y_plus[method]), result.shape[:2]))

               
        # Creates DataFrame.
    df = pd.DataFrame(out_dict) 
    df_plus = pd.DataFrame(out_dict_plus) 

    return df, df_plus


In [5]:
arr = np.arange(0, 200, 1)
arr = arr.reshape((10, 10, 2))
print(np.average(arr, axis=1))

[[  9.  10.]
 [ 29.  30.]
 [ 49.  50.]
 [ 69.  70.]
 [ 89.  90.]
 [109. 110.]
 [129. 130.]
 [149. 150.]
 [169. 170.]
 [189. 190.]]


In [6]:
r, r_p = extractToDf(results)


In [7]:
from tabulate import tabulate

In [8]:
print(tabulate(r, headers='keys', tablefmt='psql'))

+----+------------+-------------+--------------+---------+--------------+------------------+----------------+---------------------+-------------------------+-------------------------+-----------------------+----------------------+----------------------+-----------------------+-----------------------+----------------------+----------------------+--------------------------+--------------------------+----------------------+----------------------+--------------------------+--------------------------+-----------------------+---------------------------+---------------------------+-------------------------+------------------------+------------------------+-------------------------+-------------------------+------------------------+------------------------+----------------------------+----------------------------+------------------------+------------------------+----------------------------+----------------------------+------------------------+----------------------------+---------------------

In [9]:
print(tabulate(r_p, headers='keys', tablefmt='psql'))

+----+------------+-------------+--------------+---------+----------------------+----------------------+-----------------------+-----------------------+----------------------+----------------------+--------------------------+--------------------------+----------------------+----------------------+--------------------------+--------------------------+------------------------+------------------------+-------------------------+-------------------------+------------------------+------------------------+----------------------------+----------------------------+------------------------+------------------------+----------------------------+----------------------------+-------------------------+-------------------------+--------------------------+--------------------------+-------------------------+-------------------------+-----------------------------+-----------------------------+-------------------------+-------------------------+-----------------------------+-----------------------------+

accuracy analysis:

In [10]:
def getParamsTranslation(df):
    methods = ["normal", "template", "frequency", "SIFT", "ORB", "AKAZE", "KAZE"]
    direction = ["up", "right"]
    param = ["x_avg", "y_avg", "failed"]

    keys = ["original", "sub_width", "sub_height", "shape", "up_true", "up_relative", "up_expect", "right_true", "right_relative", "right_expect"]

    for key in r.keys():
        if any(d in key for d in direction):
            if any(m in key for m in methods):
                if any(k in key for k in param):
                    keys.append(key)

    print(keys)
    return df[keys]
getParamsTranslation(r)

['original', 'sub_width', 'sub_height', 'shape', 'up_true', 'up_relative', 'up_expect', 'right_true', 'right_relative', 'right_expect', 'normal_right_failed', 'normal_right_x_avg', 'normal_right_y_avg', 'template_right_failed', 'template_right_x_avg', 'template_right_y_avg', 'frequency_right_failed', 'frequency_right_x_avg', 'frequency_right_y_avg', 'SIFT_right_failed', 'SIFT_right_x_avg', 'SIFT_right_y_avg', 'ORB_right_failed', 'ORB_right_x_avg', 'ORB_right_y_avg', 'AKAZE_right_failed', 'AKAZE_right_x_avg', 'AKAZE_right_y_avg', 'KAZE_right_failed', 'KAZE_right_x_avg', 'KAZE_right_y_avg', 'normal_up_failed', 'normal_up_x_avg', 'normal_up_y_avg', 'template_up_failed', 'template_up_x_avg', 'template_up_y_avg', 'frequency_up_failed', 'frequency_up_x_avg', 'frequency_up_y_avg', 'SIFT_up_failed', 'SIFT_up_x_avg', 'SIFT_up_y_avg', 'ORB_up_failed', 'ORB_up_x_avg', 'ORB_up_y_avg', 'AKAZE_up_failed', 'AKAZE_up_x_avg', 'AKAZE_up_y_avg', 'KAZE_up_failed', 'KAZE_up_x_avg', 'KAZE_up_y_avg']


,original,sub_width,sub_height,shape,up_true,up_relative,up_expect,right_true,right_relative,right_expect,...,SIFT_up_y_avg,ORB_up_failed,ORB_up_x_avg,ORB_up_y_avg,AKAZE_up_failed,AKAZE_up_x_avg,AKAZE_up_y_avg,KAZE_up_failed,KAZE_up_x_avg,KAZE_up_y_avg
0,Dallas,250,250,"[4, 4]","[200.0, 0.0]","[210.0, 0.0]","[0.0, 0.0]","[0.0, 200.0]","[0.0, 190.0]","[0.0, 0.0]",...,-11.666667,12,0.0,-20.0,12,0.0,-20.0,11,0.0,-18.333333
1,Dallas,400,400,"[2, 2]","[320.0, 0.0]","[330.0, 0.0]","[0.0, 0.0]","[0.0, 320.0]","[0.0, 310.0]","[0.0, 0.0]",...,0.000000,0,0.0,0.0,2,0.0,-20.0,1,0.0,-10.000000
2,Dallas,500,500,"[2, 2]","[400.0, 0.0]","[410.0, 0.0]","[0.0, 0.0]","[0.0, 400.0]","[0.0, 390.0]","[0.0, 0.0]",...,0.000000,0,0.0,0.0,2,0.0,-20.0,2,0.0,-20.000000
3,Finnmark,250,250,"[4, 4]","[200.0, 0.0]","[210.0, 0.0]","[0.0, 0.0]","[0.0, 200.0]","[0.0, 190.0]","[0.0, 0.0]",...,0.000000,12,0.0,-20.0,12,0.0,-20.0,8,0.0,-13.333333
4,Finnmark,400,400,"[2, 2]","[320.0, 0.0]","[330.0, 0.0]","[0.0, 0.0]","[0.0, 320.0]","[0.0, 310.0]","[0.0, 0.0]",...,0.000000,0,0.0,0.0,2,0.0,-20.0,1,0.0,-10.000000
5,Finnmark,500,500,"[2, 2]","[400.0, 0.0]","[410.0, 0.0]","[0.0, 0.0]","[0.0, 400.0]","[0.0, 390.0]","[0.0, 0.0]",...,0.000000,0,0.0,0.0,2,0.0,-20.0,0,0.0,0.000000
6,Hellas,250,250,"[4, 4]","[200.0, 0.0]","[210.0, 0.0]","[0.0, 0.0]","[0.0, 200.0]","[0.0, 190.0]","[0.0, 0.0]",...,-15.000000,12,0.0,-20.0,12,0.0,-20.0,12,0.0,-20.000000
7,Hellas,400,400,"[2, 2]","[320.0, 0.0]","[330.0, 0.0]","[0.0, 0.0]","[0.0, 320.0]","[0.0, 310.0]","[0.0, 0.0]",...,0.000000,0,0.0,0.0,1,0.0,-10.0,0,0.0,0.000000
8,Hellas,500,500,"[2, 2]","[400.0, 0.0]","[410.0, 0.0]","[0.0, 0.0]","[0.0, 400.0]","[0.0, 390.0]","[0.0, 0.0]",...,-10.000000,1,0.0,-10.0,2,0.0,-20.0,2,0.0,-20.000000
9,Istanbul,250,250,"[4, 4]","[200.0, 0.0]","[210.0, 0.0]","[0.0, 0.0]","[0.0, 200.0]","[0.0, 190.0]","[0.0, 0.0]",...,0.000000,12,0.0,-20.0,12,0.0,-20.0,9,0.0,-15.000000


In [11]:
def getAverageTranslation(df, methods=None, directions=None, width=None):
    if directions is None:
        directions = ["right", "up"]
    if methods is None:
        methods = ["normal", "template", "frequency", "SIFT", "ORB", "AKAZE", "KAZE"]
    if width is not None:
        df = df[df["sub_width"].isin(width)]
   

    res = {
        "method": methods
    }

    for image in df["original"]:
        if image not in res:
            res[image] = []
        for method in methods:
            temp_r = []
            
            for direction in directions:
                


                df_o = df[df["original"] == image]
                choice = np.where(np.array(df_o[f"{method}_{direction}_failed"])[0] == False)
                # if image == "Trondheim":
                #     a_a = []
                #     b_b = []
                #     for a, b in zip(choice[0], choice[1]):
                #         if a != max(choice[0]):
                #             a_a.append(a)
                #             b_b.append(b)
                #     choice = (np.array(a_a), np.array(b_b))
                #     print(np.where(np.array(df_o[f"{method}_{direction}_x_avg"])[0] > 0))
                l = np.sum(np.array(df_o[f"{method}_{direction}_failed"])[0])

                if len(choice[0]) == 0 and len(choice[1]) == 0:
                    x = 0
                    y = 0
                else:
                    x = np.array(df_o[f"{method}_{direction}_x_avg"])[0][choice]
                    y = np.array(df_o[f"{method}_{direction}_y_avg"])[0][choice]
                x_avg = np.average(x)
                y_avg = np.average(y)
                temp_r.append([x_avg, y_avg, np.sqrt(x_avg**2 + y_avg**2), l/len(np.array(df_o[f"{method}_{direction}_x_avg"])[0].flatten())])
                print(np.array(df_o[f"{method}_{direction}_x_avg"]))
            res[image].append(temp_r)
           
    return res
res = getAverageTranslation(getParamsTranslation(r), ["normal", "frequency", "SIFT", "ORB", "AKAZE", "KAZE"], None, [500])
print(tabulate(res, headers='keys', tablefmt='psql'))


['original', 'sub_width', 'sub_height', 'shape', 'up_true', 'up_relative', 'up_expect', 'right_true', 'right_relative', 'right_expect', 'normal_right_failed', 'normal_right_x_avg', 'normal_right_y_avg', 'template_right_failed', 'template_right_x_avg', 'template_right_y_avg', 'frequency_right_failed', 'frequency_right_x_avg', 'frequency_right_y_avg', 'SIFT_right_failed', 'SIFT_right_x_avg', 'SIFT_right_y_avg', 'ORB_right_failed', 'ORB_right_x_avg', 'ORB_right_y_avg', 'AKAZE_right_failed', 'AKAZE_right_x_avg', 'AKAZE_right_y_avg', 'KAZE_right_failed', 'KAZE_right_x_avg', 'KAZE_right_y_avg', 'normal_up_failed', 'normal_up_x_avg', 'normal_up_y_avg', 'template_up_failed', 'template_up_x_avg', 'template_up_y_avg', 'frequency_up_failed', 'frequency_up_x_avg', 'frequency_up_y_avg', 'SIFT_up_failed', 'SIFT_up_x_avg', 'SIFT_up_y_avg', 'ORB_up_failed', 'ORB_up_x_avg', 'ORB_up_y_avg', 'AKAZE_up_failed', 'AKAZE_up_x_avg', 'AKAZE_up_y_avg', 'KAZE_up_failed', 'KAZE_up_x_avg', 'KAZE_up_y_avg']


C:\Users\sondr\AppData\Local\Temp\ipykernel_18896\1013103365.py:25: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  choice = np.where(np.array(df_o[f"{method}_{direction}_failed"])[0] == False)


IndexError: invalid index to scalar variable.

In [12]:
# print(f"Method: & {' & '.join([f'multicolumn{{2}}{{c}}{{{m}}}' for m in res][1:])}")
print(f"Method: & {' & '.join([m for m in res][1:])}\\\\\\hline")
# print("& " + ' & &'.join([f'' for m in res]))
for i in range(len(res["method"])):
    row = str(res["method"][i])
    for image in res:
        if image == "method":
            continue
        x = (res[image][i][0][0] + res[image][i][1][0] )/2
        y = (res[image][i][0][1] + res[image][i][1][1])/2 
        dist = np.sqrt(x**2 + y**2 )
        if 100*(res[image][i][0][-1] + res[image][i][1][-1])/2 == 100:
            row += f" & -"
        else:
            if dist == 0:
                row += f" & {dist:.1f}"
            else:
                row += f" & {dist:.3f}"
            if res[image][i][0][-1] + res[image][i][1][-1] > 0:
                row += "*"
    print(row + "\\\\")

print(f"Method: & {' & '.join([m for m in res][1:])}\\\\\\hline")
# print("& " + ' & &'.join([f'' for m in res]))
for i in range(len(res["method"])):
    row = str(res["method"][i])
    for image in res:
        if image == "method":
            continue
        x = (res[image][i][0][0] + res[image][i][1][0] )/2
        y = (res[image][i][0][1] + res[image][i][1][1])/2 
        dist = np.sqrt(x**2 + y**2 )
        p = 100*(res[image][i][0][-1] + res[image][i][1][-1])/2
        if p == 0:
            row += f" & {p:.1f}\\%"
        elif p == 100:
            row += f" & {int(p)}\\%"
        elif p > 10:
            row += f" & {p:.1f}\\%"
        else:
            row += f" & {p:.2f}\\%"

            
    print(row + "\\\\")

NameError: name 'res' is not defined

In [13]:

method = "a" #["SIFT"]
direction = ["up", "right"]
param = ["x_avg", "y_avg", "failed"]#, "x_mode", "y_mode"]

keys = ["original", "sub_width", "sub_height", "shape", "up_true", "up_relative", "up_expect", "right_true", "right_relative", "right_expect"]

for key in r.keys():
    if any(d in key for d in direction):
        if any(m in key for m in method):
            if any(k in key for k in param):
                keys.append(key)
        
print(tabulate(r[keys], headers='keys', tablefmt='psql'))

+----+------------+-------------+--------------+---------+-------------+---------------+-------------+--------------+------------------+----------------+-----------------------+----------------------+----------------------+-------------------------+------------------------+------------------------+--------------------------+-------------------------+-------------------------+---------------------+--------------------+--------------------+--------------------+-------------------+-------------------+----------------------+---------------------+---------------------+---------------------+--------------------+--------------------+--------------------+-------------------+-------------------+----------------------+---------------------+---------------------+-----------------------+----------------------+----------------------+------------------+-----------------+-----------------+-----------------+----------------+----------------+-------------------+------------------+------------------+---

Timing:

In [14]:
methods = ["normal", "template", "frequency", "SIFT", "ORB", "AKAZE", "KAZE"]
direction = ["up", "right"]
param = ["time", "failed"]
keys = ["original", "sub_width", "shape"]
for key in r.keys():
    if any(d in key for d in direction):
        if any(m in key for m in methods):
            if any(k in key for k in param):
                keys.append(key)

print(tabulate(r[keys], headers='keys', tablefmt='psql'))
    


+----+------------+-------------+---------+---------------------+-------------------------+-------------------------+-----------------------+-----------------------+---------------------------+---------------------------+-------------------------+------------------------+----------------------------+----------------------------+--------------------------+-------------------+-----------------------+-----------------------+---------------------+------------------+----------------------+----------------------+--------------------+--------------------+------------------------+------------------------+----------------------+-------------------+-----------------------+-----------------------+---------------------+------------------+----------------------+----------------------+--------------------+--------------------+------------------------+------------------------+----------------------+---------------------+-------------------------+-------------------------+-----------------------+-----

In [15]:
sub_width = [250, 400, 500]

def getAverageTime(df, methods=None, directions=None, width=None):
    if directions is None:
        directions = ["right", "up"]
    if methods is None:
        methods = ["normal", "template", "frequency", "SIFT", "ORB", "AKAZE", "KAZE"]
    if width is not None:
        df = df[df["sub_width"].isin(width)]
   

    avg_time = np.array([])
    l = 0
    for method in methods:
        for direction in directions:

            choice = np.where(np.array(df[f"{method}_{direction}_failed"]) == 0)
            l += np.sum(np.array(df[f"{method}_{direction}_failed"]))
            avg_time = np.concatenate((avg_time, np.array(df[f"{method}_{direction}_time"])[choice]))

    print(f"Average time of methods {methods} in directions {directions} with widths {width}")
    return avg_time, l

r_out = {
    "sub_width": [],
    "method": [],
    "time_up": [],
    "time_right": [],
}

methods = ["normal", "frequency", "SIFT", "ORB", "AKAZE", "KAZE"]
directions = ["right", "up"]


out_methods = []
out_average = []
out_var = []
out_fail = []

for width in [250, 400, 500]:
    for text, method in zip(["Intensity", "Frequency", "SIFT", "ORB", "AKAZE", "KAZE"], methods):
        print(text, method)
        arr, length = getAverageTime(r[keys], [method], None, [width])
        if width == 250:
            error_rate = length/(4*3*2*6)
            print((4*3*2*6))
        else:
            error_rate = length/(4*2*6)
            print(4*2*6)
        out_methods.append(text)
        out_average.append(f"{np.average(arr):.4f}s")
        out_var.append(f"{np.std(arr):.4f}s")
        out_fail.append(f"{100*(error_rate):.2f}\%")

    print()
    for a, b, c, d in zip(out_methods, out_average, out_var, out_fail):
        print(f"{a} & {b} & {c} & {d} \\\\")
    print()

print(out_average)

Intensity normal
Average time of methods ['normal'] in directions ['right', 'up'] with widths [250]
144
Frequency frequency
Average time of methods ['frequency'] in directions ['right', 'up'] with widths [250]
144
SIFT SIFT
Average time of methods ['SIFT'] in directions ['right', 'up'] with widths [250]
144
ORB ORB
Average time of methods ['ORB'] in directions ['right', 'up'] with widths [250]
144
AKAZE AKAZE
Average time of methods ['AKAZE'] in directions ['right', 'up'] with widths [250]
144
KAZE KAZE
Average time of methods ['KAZE'] in directions ['right', 'up'] with widths [250]
144

Intensity & 0.1783s & 0.0915s & 0.00\% \\
Frequency & 0.0064s & 0.0026s & 0.00\% \\
SIFT & 0.0129s & 0.0025s & 37.50\% \\
ORB & nans & nans & 100.00\% \\
AKAZE & nans & nans & 100.00\% \\
KAZE & nans & nans & 88.19\% \\

Intensity normal
Average time of methods ['normal'] in directions ['right', 'up'] with widths [400]
48
Frequency frequency
Average time of methods ['frequency'] in directions ['right',

c:\anaconda3\envs\machinevision\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\anaconda3\envs\machinevision\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\anaconda3\envs\machinevision\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\anaconda3\envs\machinevision\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\anaconda3\envs\machinevision\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\anaconda3\envs\machinevision\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg 

In [16]:
from coordinate_transform import relativeToAbsolute, generateOutput
def showResults_save(images_results, method):
    """
    Shows the results from the images in results, both x and y with the desired method.
    
    """
    
    res_x = images_results["result_x"]
    res_y = images_results["result_y"]
    image = np.zeros((res_x.shape[0], res_x.shape[1]+1, res_x[0, 0]["image1"].shape[0], res_x[0, 0]["image1"].shape[1]), np.uint8)
    
    print(res_x.shape)
    print(res_y.shape)
    x_coords = np.zeros((res_x.shape[0], res_x.shape[1] + 1, 2), np.int16)
    for i in range(res_x.shape[0]):
        for j in range(res_x.shape[1]):
            image[i, j] = res_x[i, j]["image1"]
            if j == res_x.shape[1] - 1:
                image[i, j+1] = res_x[i, j]["image2"]

            x_coords[i, j+1] = res_x[i, j]["relative position"] - res_x[i, j][method].getResult()

   
    y_coords = np.zeros((res_y.shape[0] + 1, res_y.shape[1], 2), np.int16)
    for i in range(res_y.shape[0]):
        for j in range(res_y.shape[1]):
            y_coords[i+1, j] = res_y[i, j]["relative position"] - res_y[i, j][method].getResult()



    x_coords[1:, 0] = y_coords[1:, 0]

    y_coords[0, 1:] = x_coords[0, 1:]

    x_coords = relativeToAbsolute(x_coords, "x_normal")
    y_coords = relativeToAbsolute(y_coords, "y_normal")
    img_x = generateOutput(image, x_coords)

    cv.imwrite(f"{images_results['name']}_{images_results['split']}_{method}_x.jpg", img_x.astype(np.uint8))


    img_y = generateOutput(image, y_coords)
    cv.imwrite(f"{images_results['name']}_{images_results['split']}_{method}_y.jpg", img_y.astype(np.uint8))

  

def findImageResult_save(results, folder_name, split_x, split_y = None, method="normal"):
    """
    From a result list with dicionaries for the folder tests print the resulting image

    results: list of dictionaries with the result of the split of that dict
    folder_name: name of the folder that was stitched, will look for in results["name"] == folder_name
    split_x: the split in the x-direction of the images,
    split_y: split in the y-direction of the images, if None split_y = split_x
    method: the method to plot for "normal", "template" "SIFT" etc. 
    
    """

    split_y = split_x if split_y is None else split_y
    goal_dict = None
    for d in results:
        if d["name"] == folder_name and d["split"] == split_x:
            goal_dict = d        
            break
    if goal_dict is None:
        print(f"Did not find {folder_name} with {split_x}, {split_y} in result")
        return None

    showResults_save(goal_dict, method)

In [17]:
for name in names:
    for method in  ["normal", "frequency", "SIFT", "ORB", "AKAZE", "KAZE"]:
        for widht in [250, 500]:
            findImageResult_save(results, name, widht, None, method)

(4, 3)
(3, 4)
[0 0] [200   0]
[200   0] [200   0]
[400   0] [200   0]
[  0 200] [200   0]
[200 200] [200   0]
[400 200] [200   0]
[  0 400] [200   0]
[200 400] [200   0]
[400 400] [200   0]
[  0 600] [200   0]
[200 600] [200   0]
[400 600] [200   0]
[0 0]
[  0 200]
[  0 400]
[  0 600]
[200   0]
[200 600]
[400   0]
[400 600]
[600   0]
[600 200]
[600 400]
[600 600]
out_dimensions is max: 1020, min: 142
[0 0]
[  0 200]
[  0 400]
[  0 600]
[200   0]
[200 600]
[400   0]
[400 600]
[600   0]
[600 200]
[600 400]
[600 600]
out_dimensions is max: 1020, min: 142
(2, 1)
(1, 2)
[0 0] [400   0]
[  0 400] [400   0]
[0 0]
[  0 400]
[400   0]
[400 400]
out_dimensions is max: 1020, min: 142
[0 0]
[  0 400]
[400   0]
[400 400]
out_dimensions is max: 1020, min: 142
(4, 3)
(3, 4)
[0 0] [200   0]
[200   0] [200   0]
[400   0] [200   0]
[  0 200] [200   0]
[200 200] [200   0]
[400 200] [200   0]
[  0 400] [200   0]
[200 400] [200   0]
[400 400] [200   0]
[  0 600] [200   0]
[200 600] [200   0]
[400 600] [200

In [ ]:
name = poopoo
width = 500
findImageResult(results, name, width, None, "normal")
findImageResult(results, name, width, None, "frequency")
findImageResult(results, name, width, None, "SIFT")
findImageResult(results, name, width, None, "ORB")
findImageResult(results, name, width, None, "AKAZE")
findImageResult(results, name, width, None, "KAZE")
name = "Trondheim"
findImageResult(results, name, width, None, "normal")
findImageResult(results, name, width, None, "frequency")
findImageResult(results, name, width, None, "SIFT")
findImageResult(results, name, width, None, "ORB")
findImageResult(results, name, width, None, "AKAZE")
findImageResult(results, name, width, None, "KAZE")
name = "Hellas"
findImageResult(results, name, width, None, "normal")
findImageResult(results, name, width, None, "frequency")
findImageResult(results, name, width, None, "SIFT")
findImageResult(results, name, width, None, "ORB")
findImageResult(results, name, width, None, "AKAZE")
findImageResult(results, name, width, None, "KAZE")
name = "Istanbul"
findImageResult(results, name, width, None, "normal")
findImageResult(results, name, width, None, "frequency")
findImageResult(results, name, width, None, "SIFT")
findImageResult(results, name, width, None, "ORB")
findImageResult(results, name, width, None, "AKAZE")
findImageResult(results, name, width, None, "KAZE")
name = "Finnmark"
findImageResult(results, name, width, None, "normal")
findImageResult(results, name, width, None, "frequency")
findImageResult(results, name, width, None, "SIFT")
findImageResult(results, name, width, None, "ORB")
findImageResult(results, name, width, None, "AKAZE")
findImageResult(results, name, width, None, "KAZE")
name = "Tromsoe"
findImageResult(results, name, width, None, "normal")
findImageResult(results, name, width, None, "frequency")
findImageResult(results, name, width, None, "SIFT")
findImageResult(results, name, width, None, "ORB")
findImageResult(results, name, width, None, "AKAZE")
findImageResult(results, name, width, None, "KAZE")
# folder = getFolderName(name, 250)
# metadata_filename = f"{folder}\metadata.txt"
# images, max_r, max_c = readTileConfigurationCoordinates(metadata_filename, indecies=True)
# dallas_dic = None
# for d in results:
#     if d["name"] == name and d["split"] == 250:
#         dallas_dic = d        
#         break

# #keep on going toworrow with this,


In [ ]:
import numpy as np
from tifffile import astype



def relativeToAbsolute_scrap(coords, direction, img_shape):
    """
    Converts the coordinates from relative (every single coordinat erelative to the last)
    to absolute (absolute in large grid)

    coords
    direction = "x_normal" "x_alternate" "y_normal" "y_alternate"
    
    """

    prev_row = None
    prev_col = None

    row = 0
    col = 0
    
    if direction == "x_normal":
        end = coords.shape[:2]
    elif direction == "x_alternate":
        end = coords.shape[:2] if coords.shape[0] %2 != 0 else (coords.shape[0], 0)
    elif direction == "y_normal":
        end = coords.shape[:2]
    elif direction == "y_alternate":
        end = coords.shape[:2] if coords.shape[1] % 2 != 0 else (0, coords.shape[1])

    else:
        end = coords.shape[:2]

    while row != end[0] - 1 or col != end[1] - 1:
        if prev_row is not None or prev_col is not None:
            if direction == "x_normal":
                print( coords[prev_row, prev_col] + coords[row, col]  )
                coords[row, col] = coords[prev_row, prev_col] + coords[row, col]   
            elif direction == "x_alternate":
                if row % 2 == 0 or col == prev_col:
                    coords[row, col] = coords[prev_row, prev_col] + coords[row, col]   
                else:
                    coords[row, col][0] = coords[prev_row, prev_col][0] + coords[row, col][0]
                    coords[row, col][1] = coords[prev_row, prev_col][1] - coords[row, col][1]
                    
            elif direction == "y_normal":
                print( coords[prev_row, prev_col] + coords[row, col]  )
                coords[row, col] = coords[prev_row, prev_col] + coords[row, col]   
            elif direction == "y_alternate":
                if col % 2 == 0 or row == prev_row:
                    coords[col, row] = coords[prev_col, prev_row] + coords[col, row]   
                else:
                    coords[col, row][0] = coords[prev_col, prev_row][0] + coords[col, row][0]
                    coords[col, row][1] = coords[prev_col, prev_row][1] - coords[col, row][1]
        

        prev_col = col
        prev_row = row

        if direction == "x_normal":
            if col == coords.shape[1] - 1:
                prev_col = 0
                prev_row = row
                col = 0
                row += 1
            else:
                col += 1            
        elif direction == "x_alternate":
            if row % 2 == 0:
                if col == coords.shape[1] - 1:
                    row += 1
                else:
                    col += 1
            else:
                if col == 0:
                    row += 1
                else:
                    col -= 1
        elif direction == "y_normal":
            if row == coords.shape[0] - 1:
                prev_row = 0
                prev_col = col
                row = 0
                col += 1
            else:
                row += 1
        elif direction == "y_alternate":
            if col % 2 == 0:
                if row == coords.shape[0] - 1:
                    col += 1
                else:
                    row += 1
            else:
                if row == 0:
                    col += 1
                else:
                    row -= 1

    return coords   

def relativeToAbsolute(coords, direction):


    if direction == "x_normal":
        for i in range(coords.shape[0]):
            if i != 0:
                coords[i, 0] = coords[i - 1, 0] + coords[i, 0]
            for j in range(1, coords.shape[1]):
                coords[i, j] = coords[i, j - 1] + coords[i, j]

    if direction == "y_normal":
        for j in range(coords.shape[1]):
            if j != 0:
                coords[0, j] = coords[0, j-1] + coords[0, j]
            for i in range(1, coords.shape[0]):
                print(coords[i - 1, j],  coords[i, j])
                coords[i, j] = coords[i - 1, j] + coords[i, j]

    return coords




# test_arr = np.array([[[0, 0], [4, 4], [-4, 4]], [[9, 2], [3, 3], [5, 3]]])

# print(relativeToAbsolute(test_arr, "x_normal", 0))



def getOutputDimensions(coords, image_shape):
        x_maks = x_min = y_maks = y_min = 0
        rows = coords.shape[0]
        cols = coords.shape[1]
        for i in range(rows):
            for j in range(cols):

                if j != 0 and j != cols - 1 and i != 0 and i != rows-1:
                    continue
                p = coords[i, j]
                print(p)
                y = p[0] if p[0] != np.inf else 0
                x = p[1] if p[1] != np.inf else 0
                if x + image_shape[1] > x_maks:
                    x_maks = x + image_shape[1]
                if x < x_min:
                    x_min = x
                if y + image_shape[0] > y_maks:
                    y_maks = y + image_shape[0]
                if y < y_min:
                    y_min = y
        return x_maks, x_min, y_maks, y_min

def generateOutput(images, coords, distance=None, vignett_inv=None, border = False):

    if distance is None and vignett_inv is None:
        transform = lambda image: image.astype(np.uint32)
    elif distance is None:
        transform = lambda image: (image*vignett_inv).astype(np.uint32)
    elif vignett_inv is None:
        transform = lambda image: (image*distance).astype(np.uint32)
    else:
        transform = lambda image: (image*vignett_inv*distance).astype(np.uint32)


    shape = images[0, 0].shape
    x_maks, x_min, y_maks, y_min = getOutputDimensions(coords, shape)

    out = np.zeros((abs(y_min) + y_maks, abs(x_min) + x_maks), dtype=np.uint32)
    if distance is not None:
        out_d = out.copy()

    for i in range(coords.shape[0]):
        for j in range(coords.shape[1]):
            dy = coords[i,j][0] if coords[i,j][1] != np.inf else 0
            dx = coords[i,j][1] if coords[i,j][0] != np.inf else 0
            f_y = y_maks - dy - shape[0]
            f_x = abs(x_min) + dx
            # print(dx, dy)
            out[f_y:f_y + shape[0], f_x:f_x + shape[1]] += transform(images[i, j])
            cv.imshow(f"{i}, {j}", images[i, j])
            if distance is not None:
                out_d[f_y:f_y + shape[0], f_x:f_x + shape[1]] += distance
            if border:
                out[f_y-10, f_x - 100: f_x + shape[1] + 100] = 255
                out[f_y+10, f_x - 100: f_x + shape[1] + 100] = 255
                out[f_y - 100:f_y + shape[0] + 100, f_x-10] = 255
                out[f_y - 100:f_y + shape[0] + 100:, f_x+10] = 255
                if distance is not None:
                    out_d[f_y-10, f_x - 100: f_x + shape[1] + 100] = 1
                    out_d[f_y+10, f_x - 100: f_x + shape[1] + 100] = 1
                    out_d[f_y - 100:f_y + shape[0] + 100, f_x-10] = 1
                    out_d[f_y - 100:f_y + shape[0] + 100:, f_x+10] = 1
    if distance is not None:
        out = out / out_d
    print(f"out_dimensions is max: {np.amax(out)}, min: {np.amin(out)}")
    cv.waitKey(0)
    return out.astype(np.uint8)

import cv2 as cv
def showResults(images_results, method):
    """
    Shows the results from the images in results, both x and y with the desired method.
    
    """
    
    res_x = images_results["result_x"]
    res_y = images_results["result_y"]
    image = np.zeros((res_x.shape[0], res_x.shape[1]+1, res_x[0, 0]["image1"].shape[0], res_x[0, 0]["image1"].shape[0]), np.uint8)
    
    print(res_x.shape)
    print(res_y.shape)
    x_coords = np.zeros((res_x.shape[0], res_x.shape[1] + 1, 2), np.int16)
    # cv.imshow("1", res_x[-1, 0]["image1"])
    # cv.imshow("2", res_x[-1, 0]["image2"])
    # cv.waitKey(0)
    # cv.imshow("1", res_x[0, -1]["image1"])
    # cv.imshow("2", res_x[0, -1]["image2"])
    for i in range(res_x.shape[0]):
        for j in range(res_x.shape[1]):
            image[i, j] = res_x[i, j]["image1"]
            if j == res_x.shape[1] - 1:
                image[i, j+1] = res_x[i, j]["image2"]

            x_coords[i, j+1] = res_x[i, j]["relative position"] - res_x[i, j][method].getResult()

   
    y_coords = np.zeros((res_y.shape[0] + 1, res_y.shape[1], 2), np.int16)
    for i in range(res_y.shape[0]):
        for j in range(res_y.shape[1]):
            y_coords[i+1, j] = res_y[i, j]["relative position"] - res_y[i, j][method].getResult()



    x_coords[1:, 0] = y_coords[1:, 0]

    y_coords[0, 1:] = x_coords[0, 1:]

    x_coords = relativeToAbsolute(x_coords, "x_normal")
    y_coords = relativeToAbsolute(y_coords, "y_normal")
    img_x = generateOutput(image, x_coords)
    cv.imshow(f"x_dir_{method}", img_x)
    cv.waitKey(0)

    print(y_coords)
    img_y = generateOutput(image, y_coords)
    cv.imshow(f"y_dir_{method}", img_y)
    cv.waitKey(0)

    cv.destroyAllWindows()


def findImageResult(results, folder_name, split_x, split_y = None, method="normal"):
    """
    From a result list with dicionaries for the folder tests print the resulting image

    results: list of dictionaries with the result of the split of that dict
    folder_name: name of the folder that was stitched, will look for in results["name"] == folder_name
    split_x: the split in the x-direction of the images,
    split_y: split in the y-direction of the images, if None split_y = split_x
    method: the method to plot for "normal", "template" "SIFT" etc. 
    
    """

    split_y = split_x if split_y is None else split_y
    goal_dict = None
    for d in results:
        if d["name"] == folder_name and d["split"] == split_x:
            goal_dict = d        
            break
    if goal_dict is None:
        print(f"Did not find {folder_name} with {split_x}, {split_y} in result")
        return None

    showResults(goal_dict, method)



In [ ]:
findImageResult(results, name, width, None, "normal")